In [10]:
import pandas as pd
import json

def preprocess_argotario(input_path):
    def get_gold_label(row):
        """
        output:
        0/1/2 for is_gold_example, fallacy label
        """
        try:
            intent = row['Intended Fallacy'].strip()
            voted = row['Voted Fallacy'].strip()
        except:
            print(row['Intended Fallacy'])
            print(row['Voted Fallacy'])
        n_votes = int(row['Number of Votes'])
        if n_votes >= 5:
            if (intent==voted) or (voted=="-"):
                return 1, intent
            elif (intent != voted) and (voted!="-"):
                return 2, voted + "|" + intent
        else:
            if (intent==voted) or (voted=="-"):
                return 0, intent
            elif (intent != voted) and (voted!="-"):
                return 0, intent + "|" + voted
    def get_qa(row):
        return f'''A: {row['Topic'].strip()}\nB: {row['Text'].strip()}'''
    
    df = pd.read_csv(input_path,sep='\t')
    df = df.drop(df[(df['Text'].isna()) | (df['Topic'].isna())].index)
    df['Intended Fallacy'] = df['Intended Fallacy'].apply(lambda x: 'Appeal to False Authority' if x == 'Irrelevant Authority' else x)
    df['Voted Fallacy'] = df['Voted Fallacy'].apply(lambda x: 'Appeal to False Authority' if x == 'Irrelevant Authority' else x)
    df[['is_gold', 'label']] = df[['Intended Fallacy', 'Voted Fallacy', 'Number of Votes']].apply(get_gold_label, axis=1, result_type="expand")
    df['text'] = df[['Topic', 'Text']].apply(get_qa, axis=1)
    df['stance'] = df['Stance'].apply(lambda x: 1 if x=='pro' else 0)
    df = df.drop(columns=['Stance', 'Topic', 'Text', 'Row number', 'Mongo ID', 'Author', 'Intended Fallacy', 'Voted Fallacy', 'Number of Votes'])
    df = df[['text', 'label', 'stance', 'is_gold']]
    print(df['is_gold'].value_counts())
    print(df['label'].value_counts())
    print(df['stance'].value_counts())
    print(df[df['is_gold'] == 1]['label'].value_counts())
    js = df.to_json(orient="records", indent=4)#lines=True
    
    with open('data.json', 'w') as f:
        f.write(js)
        f.close()
    return
preprocess_argotario('./arguments-en-2018-01-15.tsv')


is_gold
0    969
1    312
2     42
Name: count, dtype: int64
label
No Fallacy                                        407
Appeal to Emotion                                 223
Red Herring                                       165
Ad Hominem                                        152
Hasty Generalization                              149
Appeal to False Authority                         142
No Fallacy|Red Herring                             14
Appeal to Emotion|No Fallacy                        7
Ad Hominem|No Fallacy                               7
Appeal to Emotion|Red Herring                       6
No Fallacy|Appeal to False Authority                5
Appeal to False Authority|Red Herring               5
No Fallacy|Hasty Generalization                     5
Ad Hominem|Red Herring                              5
Red Herring|No Fallacy                              4
Red Herring|Hasty Generalization                    3
Hasty Generalization|Ad Hominem                     3
Appeal to False

In [25]:
# df = preprocess_argotario('./arguments-en-2018-01-15.tsv')
# df['Intended Fallacy'].value_counts()
# df['Number of Votes'].value_counts()
# df[df['Number of Votes']>=5.]['Number of Votes'].value_counts()
# df1 = df[df['Number of Votes']>=5.]
# df1
# df1[df1['Intended Fallacy'] == df1['Voted Fallacy']]
# df[df['Intended Fallacy']!='No Fallacy']['Number of Votes'].value_counts()
# #df[df['Voted Fallacy']!='No Fallacy']['Number of Votes'].value_counts()
# df.columns
# df['Stance'].value_counts()
df = pd.read_csv('./arguments-en-2018-01-15.tsv',sep='\t')
df[df['Intended Fallacy'].isna()]

,Row number,Mongo ID,Author,Topic,Stance,Intended Fallacy,Voted Fallacy,Number of Votes,Text
173,Whereas this soccer-player gets like 15 euros ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,Read more at http://www.jamieoliver.com/news-a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
679,A daycare-service would take some pressure of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
731,'Animals have the right to equal consideration...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
732,"However, animals don’t always have the same ri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1184,"If you start regulating private things, like s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
